# Practicing with merge

In [2]:
import numpy as np
import pandas as pd

In [8]:
# PRACTICING MERGE -- ignore this cell
sample_school = pd.DataFrame({'BEDS Code':[0,1,2,3,4,5,1,2,3,1],
                              'Other Field': list('abcdefghij'),
                              'Another Field':[10,11,12,13,14,15]+[np.nan]*4})
sample_latlon = pd.DataFrame({'SED CODE':[1,2,3], 'latlon':['one','two','three']})

new = pd.merge(sample_school, sample_latlon, left_on=['BEDS Code'],right_on=['SED CODE'], how='left')
new

,Another Field,BEDS Code,Other Field,SED CODE,latlon
0,10,0,a,NaN,NaN
1,11,1,b,1,one
2,12,2,c,2,two
3,13,3,d,3,three
4,14,4,e,NaN,NaN
5,15,5,f,NaN,NaN
6,NaN,1,g,1,one
7,NaN,2,h,2,two
8,NaN,3,i,3,three
9,NaN,1,j,1,one


In [9]:
# Take a look at the resulting data/missing values
print('... {} unique schools,'.format(len(new['BEDS Code'].unique())))
schools_missing_latlon = new[new['latlon'].isnull()]['BEDS Code'].unique()
print('... of which {} are missing lat/long'.format(len(schools_missing_latlon)))

... 6 unique schools,
... of which 3 are missing lat/long


# Practicing with fillling na
GOAL: to use this to fill in missing boroughs

In [15]:
# create dictionary for filling 'Another Field'
d = new[new['Another Field'].notnull()][['BEDS Code','Another Field']].to_dict()
d

{'Another Field': {0: 10.0, 1: 11.0, 2: 12.0, 3: 13.0, 4: 14.0, 5: 15.0},
 'BEDS Code': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5}}

In [14]:
map_vals = {d['BEDS Code'][idx]: d['Another Field'][idx] for idx in d['Another Field'].keys()}
map_vals

{0: 10.0, 1: 11.0, 2: 12.0, 3: 13.0, 4: 14.0, 5: 15.0}

In [26]:
new['Another Field'] = map(map_vals,new['BEDS Code'])

In [18]:
new

,Another Field,BEDS Code,Other Field,SED CODE,latlon
0,<map object at 0x10784d8d0>,0,a,NaN,NaN
1,<map object at 0x10784d8d0>,1,b,1,one
2,<map object at 0x10784d8d0>,2,c,2,two
3,<map object at 0x10784d8d0>,3,d,3,three
4,<map object at 0x10784d8d0>,4,e,NaN,NaN
5,<map object at 0x10784d8d0>,5,f,NaN,NaN
6,<map object at 0x10784d8d0>,1,g,1,one
7,<map object at 0x10784d8d0>,2,h,2,two
8,<map object at 0x10784d8d0>,3,i,3,three
9,<map object at 0x10784d8d0>,1,j,1,one
